In [1]:
# read stock data from website


In [1]:
import requests
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import xlwings
pd.set_option("display.float.format", str)

In [8]:
def get_nse_data(inst_type):
    url ="https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY"
    headers  = {
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"
    }

    session = requests.session()
    data  = session.get(url, headers= headers).json()['records']['data']
    oc_data = []
    for i in data:
        for j, k in i.items():
            if j == 'CE' or j =='PE':
                info = k
                info['instrumentType'] = j
                oc_data.append(info)

    df = pd.DataFrame(oc_data)
    df.expiryDate = pd.to_datetime(df.expiryDate)
    # df.to_excel('option_chain_bank.xlsx',index = False)
    df_1 = df.query(f"(strikePrice >= 38000) & (strikePrice <= 40000)  &(expiryDate == '2023-03-23') &(instrumentType =='{inst_type}')  ")
    df_1['up'] = df_1.lastPrice.diff()
    df_1['down'] = df_1.up.shift(-1)
    df_1['diff'] = abs(df_1['up']) - abs(df_1['down'])
    
    df_1 =   df_1.round(2)
    # time.sleep(120)
    return df

In [9]:
get_nse_data(inst_type = 'CE')

,strikePrice,expiryDate,underlying,identifier,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,instrumentType
0,31500,2023-06-29,BANKNIFTY,OPTIDXBANKNIFTY29-06-2023CE31500.00,0,0,0.0,0,0.0,0.0,0.0,0.0,2325,1375,50,12262.35,500,12961.25,43968.5,CE
1,33000,2023-06-29,BANKNIFTY,OPTIDXBANKNIFTY29-06-2023CE33000.00,0,0,0.0,0,0.0,0.0,0.0,0.0,3050,1375,775,10692.0,500,11325.95,43968.5,CE
2,33000,2024-03-28,BANKNIFTY,OPTIDXBANKNIFTY28-03-2024PE33000.00,0,0,0.0,0,0.0,0.0,0.0,0.0,900,0,900,1.7,0,0.0,43968.5,PE
3,33000,2023-09-28,BANKNIFTY,OPTIDXBANKNIFTY28-09-2023PE33000.00,196,0,0.0,0,0.0,0.0,0.0,0.0,1325,25,25,29.05,25,40.0,43968.5,PE
4,34000,2023-06-29,BANKNIFTY,OPTIDXBANKNIFTY29-06-2023PE34000.00,10,0,0.0,0,0.0,0.0,0.0,0.0,5250,0,975,3.05,0,0.0,43968.5,PE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007,50500,2023-08-31,BANKNIFTY,OPTIDXBANKNIFTY31-08-2023CE50500.00,0,0,0.0,0,0.0,0.0,0.0,0.0,0,180,0,0.0,15,325.0,43968.5,CE
1008,50500,2023-08-31,BANKNIFTY,OPTIDXBANKNIFTY31-08-2023PE50500.00,0,0,0.0,0,0.0,0.0,0.0,0.0,165,0,150,4646.05,0,0.0,43968.5,PE
1009,51000,2023-06-29,BANKNIFTY,OPTIDXBANKNIFTY29-06-2023PE51000.00,38,0,0.0,4,39.27,6883.35,-196.64999999999964,-2.7775423728813506,2500,2425,50,6812.8,100,6955.7,43968.5,PE
1010,51000,2023-06-29,BANKNIFTY,OPTIDXBANKNIFTY29-06-2023CE51000.00,732,46,6.705539358600583,111,22.57,10.6,0.34999999999999964,3.4146341463414602,19425,9900,200,10.0,1000,10.55,43968.5,CE


In [4]:
import time 
while(True):
    req_cols = ['strikePrice','lastPrice','up','down','diff']
    df = get_nse_data(inst_type='CE')[req_cols].merge(get_nse_data(inst_type='PE')[req_cols], on = 'strikePrice')
    df 
    break

In [5]:
df

,lastPrice_x,up_x,down_x,diff_x,strikePrice,lastPrice_y,up_y,down_y,diff_y


In [15]:
import os
import pandas as pd 
df = pd.DataFrame()
for i in os.listdir("data/"):
    df = df.append(pd.read_csv(f"data/{i}"))

In [16]:
df['call_put'] = df.strike_type.str[-2:]
df = df.groupby(['Symbol','date']).call_put.value_counts().reset_index(name = 'count')
df_1 = pd.pivot_table(df, index = ['Symbol', 'date'], columns = ['call_put']).reset_index()
df_1.columns = ['Symbol', "date", "CE", "PE"]
df_1[df_1.CE.isna() | df_1.PE.isna()].to_excel('top_5_option.xlsx',index = False)